# Bloch-Torrey Equation

## Introduction

Here we solve the Bloch-Torrey equation on a unit square, with the diffusion coefficient $D(x)$, relaxation rate $R(x)$, and resonance frequency $\omega(x)$ all given as a generic functions.
The strong form of the Bloch-Torrey equation is given by

\begin{align}
    \frac{\partial u_x}{\partial t} &= \nabla \cdot (D \nabla u_x) - R u_x + \omega u_y  \quad x \in \Omega\\
    \frac{\partial u_y}{\partial t} &= \nabla \cdot (D \nabla u_y) - R u_y - \omega u_x  \quad x \in \Omega,
\end{align}

where $\vec{u}=[u_x,u_y]$ is the transverse magnetization, and $\Omega$ the domain.

We will consider homogeneous Neumann boundary conditions such that

\begin{align}
    \nabla \vec{u}(x) \cdot \hat{n} &= \vec{0}  \quad x \in \partial \Omega\\
\end{align}

where $\partial \Omega$ denotes the boundary of $\Omega$, and $\cdot$ is a tensor contraction.

The initial condition is given generically as

\begin{equation}
    \vec{u}(x,t=0) = \vec{u}_0 (x)  \quad x \in \Omega
\end{equation}


The resulting weak form is given by
\begin{align}
    \int_{\Omega} \vec{v} \cdot \vec{u}_t \, d\Omega
    &= -\int_{\Omega}
    -\vec{v} \cdot \nabla \cdot ( D \, \nabla \vec{u} ) +
    R \, \vec{v} \cdot \vec{u} -
    \omega \, \vec{v} \times \vec{u}
    \, d\Omega \\
    &= -\int_{\Omega}
    D \, \nabla \vec{v} : \nabla \vec{u} +
    R \, \vec{v} \cdot \vec{u} -
    \omega \, \vec{v} \times \vec{u}
    \, d\Omega + 
    \int_{\partial\Omega} \vec{v} \cdot (D\nabla\vec{u} \cdot \hat{n}) \, d\Gamma,
\end{align}
where $\vec{v}$ is a suitable test function.

In this notebook, we will assume homogeneous Neumann boundary conditions on all boundaries by taking $D\nabla\vec{u} \cdot \hat{n} = 0$. Therefore, the final weak form is simply
\begin{align}
    \int_{\Omega} \vec{v} \cdot \vec{u}_t \, d\Omega
    = -\int_{\Omega}
    D \, \nabla \vec{v} : \nabla \vec{u} +
    R \, \vec{v} \cdot \vec{u} -
    \omega \, \vec{v} \times \vec{u}
    \, d\Omega
\end{align}

Note: in two dimensions, the cross product is simply a scalar. However, `Tensors.jl` defines the two dimensional cross product by first extending the 2D vectors into 3D. Below, we use the symbol $\boxtimes$ to denote the scalar version, which is the same as taking the third component of the vector version

## Commented Program

Now we solve the problem in JuAFEM. What follows is a program spliced with comments.

First we load the JuAFEM package.

In [1]:
# using Distributed
# addprocs(8; restrict = true, enable_threaded_blas = true);
# @everywhere begin
#     include("init.jl")
# end

In [2]:
include("init.jl")

┌ Info: Recompiling stale cache file /home/coopar7/.julia/compiled/v0.7/Expmv.ji for Expmv [top-level]
└ @ Base loading.jl:1185
┌ Info: Recompiling stale cache file /home/coopar7/.julia/compiled/v0.7/BlochTorreyUtils.ji for BlochTorreyUtils [top-level]
└ @ Base loading.jl:1185
┌ Info: Recompiling stale cache file /home/coopar7/.julia/compiled/v0.7/BlochTorreySolvers.ji for BlochTorreySolvers [top-level]
└ @ Base loading.jl:1185


**Pack circles**: Pack circles with a specified packing density $\eta$

In [3]:
btparams = BlochTorreyParameters{Float64}(theta = pi/2, g_ratio = 0.8);

In [4]:
Dim = 2;
Ncircles = 20;

rs = rand(radiidistribution(btparams), Ncircles);
initial_circles = GreedyCirclePacking.pack(rs; iters = 100);

η = 0.8; # goal packing density
ϵ = 0.1 * btparams.R_mu; # overlap occurs when distance between circle edges is ≤ ϵ
α = 1e-1; # covariance penalty weight (enforces circular distribution)
β = 1e-6; # mutual distance penalty weight
λ = 1.0; # overlap penalty weight (or lagrange multiplier for constrained version)
w = [α, β, λ]; # vector of weights

In [5]:
minimum_signed_edge_distance(initial_circles)

-7.771561172376096e-16

In [6]:
estimate_density(initial_circles)

0.8009382373545971

In [7]:
@time outer_circles = EnergyCirclePacking.pack(initial_circles;
    autodiff = false,
    secondorder = false,
    setcallback = false,
    goaldensity = η,
    distancescale = btparams.R_mu,
    weights = w,
    epsilon = ϵ);

inner_circles = scale_shape.(outer_circles, btparams.g_ratio);

  6.869644 seconds (14.68 M allocations: 729.323 MiB, 11.12% gc time)


In [8]:
dmin = minimum_signed_edge_distance(outer_circles)
@show covariance_energy(outer_circles)
@show estimate_density(outer_circles)
@show is_any_overlapping(outer_circles)
@show (dmin, ϵ, dmin > ϵ);

covariance_energy(outer_circles) = 0.05296915395198874
estimate_density(outer_circles) = 0.8
is_any_overlapping(outer_circles) = false
(dmin, ϵ, dmin > ϵ) = (0.04897000635587584, 0.046000000000000006, true)


**Generate mesh**: Rectangular mesh with circles possibly only partly contained or completely excluded

In [9]:
#Ncircles = length(outer_circles);
#Nmin = 50; # points for average circle
#h0 = 2pi*R_mu/Nmin; # approximate scale
h0 = minimum(radius.(outer_circles))*(1-btparams.g_ratio); # fraction of size of minimum torus width
# eta = 5.0; # approx ratio between largest/smallest edges, i.e. max ≈ eta * h0
# gamma = 10.0; # max edge length of `eta * h0` occurs approx. `gamma * h0` from circle edges
# alpha = 0.7; # power law for edge length scaling between `h0` and `eta * h0`
h_min = 1.0h0; # minimum edge length
h_max = 5.0h0; # maximum edge length
h_range = 10.0h0; # distance over which h increases from h_min to h_max
h_rate = 0.6; # rate of increase of h from circle boundaries (power law; smaller = faster radial increase)

0.6

In [10]:
bdry, _ = opt_subdomain(outer_circles)

(Rectangle{2,Float64}([-1.3876, -2.95747], [2.01829, 0.448427]), 0.75)

In [11]:
@time exteriorgrids, torigrids, interiorgrids, parentcircleindices = disjoint_rect_mesh_with_tori(
    bdry, inner_circles, outer_circles, h_min, h_max, h_range, h_rate;
    maxstalliters = 500, plotgrids = false, exterior_tiling = (2, 2)
);

1/20: Interior
1/20: Annular
2/20: Interior
2/20: Annular
3/20: Interior
3/20: Annular
4/20: Interior
4/20: Annular
5/20: Interior
5/20: Annular
6/20: Interior
6/20: Annular
7/20: Interior
7/20: Annular
8/20: Interior
8/20: Annular
9/20: Interior
9/20: Annular
10/20: Interior
10/20: Annular
11/20: Interior
11/20: Annular
12/20: Interior
12/20: Annular
13/20: Interior
13/20: Annular
14/20: Interior
14/20: Annular
15/20: Interior
15/20: Annular
16/20: Interior
16/20: Annular
17/20: Interior
17/20: Annular
18/20: Interior
18/20: Annular
19/20: Interior
19/20: Annular
20/20: Interior
20/20: Annular
1/4: Exterior
2/4: Exterior
3/4: Exterior
4/4: Exterior
 40.023228 seconds (217.51 M allocations: 7.079 GiB, 14.93% gc time)


In [12]:
mxcall(:figure, 0)
simpplot.(exteriorgrids; hold = true, axis = mxaxis(bdry));
simpplot.(interiorgrids; hold = true, axis = mxaxis(bdry));
simpplot.(torigrids; hold = true, axis = mxaxis(bdry));

### Diffusion coefficient $D(x)$, relaxation rate $R(x)$, and resonance frequency $\omega(x)$

These functions are defined within `doassemble!`

### Trial and test functions
A `CellValues` facilitates the process of evaluating values and gradients of
test and trial functions (among other things). Since the problem
is a scalar problem we will use a `CellScalarValues` object. To define
this we need to specify an interpolation space for the shape functions.
We use Lagrange functions (both for interpolating the function and the geometry)
based on the reference "cube". We also define a quadrature rule based on the
same reference cube. We combine the interpolation and the quadrature rule
to a `CellScalarValues` object.

### Degrees of freedom
Next we need to define a `DofHandler`, which will take care of numbering
and distribution of degrees of freedom for our approximated fields.
We create the `DofHandler` and then add a single field called `u`.
Lastly we `close!` the `DofHandler`, it is now that the dofs are distributed
for all the elements.

Now that we have distributed all our dofs we can create our tangent matrix,
using `create_sparsity_pattern`. This function returns a sparse matrix
with the correct elements stored.

We can inspect the pattern using the `spy` function from `UnicodePlots.jl`.
By default the stored values are set to $0$, so we first need to
fill the stored values, e.g. `K.nzval` with something meaningful.

In [13]:
#using UnicodePlots
#fill!(K.nzval, 1.0)
#spy(K; height = 25)

### Boundary conditions
In JuAFEM constraints like Dirichlet boundary conditions are handled by a `ConstraintHandler`. However, here we will have no need to directly enforce boundary conditions, since Neumann boundary conditions have already been applied in the derivation of the weak form.

### Assembling the linear system
Now we have all the pieces needed to assemble the linear system, $K u = f$.
We define a function, `doassemble` to do the assembly, which takes our `cellvalues`,
the sparse matrix and our DofHandler as input arguments. The function returns the
assembled stiffness matrix, and the force vector.

In [14]:
# domains = MyelinDomain(grid, outer_circles, inner_circles, bdry, exteriorgrid, torigrids, interiorgrids; quadorder = 3, funcinterporder = 1);
myelindomains = createmyelindomains(exteriorgrids, torigrids, interiorgrids, outer_circles, inner_circles);

In [15]:
myelinprob = MyelinProblem(btparams);

In [16]:
# @time doassemble!(prob, domains);
@time map!(m -> doassemble!(m, myelinprob), myelindomains, myelindomains);

  3.127751 seconds (19.37 M allocations: 510.343 MiB, 14.75% gc time)


In [17]:
# @time factorize!(domains);
@time map!(m -> (factorize!(getdomain(m)); return m), myelindomains, myelindomains);

  0.293760 seconds (516.32 k allocations: 32.911 MiB, 3.34% gc time)


In [18]:
omegavalues = map(myelindomains) do m
    ω = BlochTorreyProblem(myelinprob, m).Omega
    return map(getnodes(getgrid(m))) do node
        ω(getcoordinates(node))
    end
end;

In [19]:
mxcall(:figure, 0)
for (m,w) in Iterators.reverse(zip(myelindomains, omegavalues))
    simpplot(getgrid(m); hold = true, axis = mxaxis(bdry), facecol = w);
end

┌ Warning: Deprecated syntax `implicit assignment to global variable `w``.
│ Use `global w` instead.
└ @ nothing none:0
┌ Warning: Loop variable `w` around In[19]:2 overwrites a variable in an enclosing scope. In the future the variable will be local to the loop instead.
└ @ nothing In[19]:2


### Solution of the differential equation system
The last step is to solve the system. First we call `doassemble`
to obtain the global stiffness matrix `K` and force vector `f`.
Then, to account for the boundary conditions, we use the `apply!` function.
This modifies elements in `K` and `f` respectively, such that
we can get the correct solution vector `u` by using `\`.

In [20]:
tspan = (0.0, 320.0e-3);
dt = 10e-3;
# saveat = tspan[1]:dt:tspan[2];
# tstops = (tspan[1] .+ dt/2 .+ dt .* (1:round(Int, (tspan[2]-tspan[1])/dt)))
u0 = Vec{2}((0.0, 1.0)); # initial pi/2 pulse

In [21]:
probs = [ODEProblem(m, interpolate(u0, m), tspan) for m in myelindomains];

In [22]:
sols = Vector{ODESolution}(undef, length(probs));

In [23]:
@time for i in eachindex(sols, probs)
    print("i = $i/$(length(sols)): ")
    A = probs[i].p[1]
    sols[i] = @time solve(probs[i], ExpokitExpmv(A; m = 30);
        dt = dt,
        reltol = 1e-4,
        callback = MultiSpinEchoCallback(tspan; TE = dt)
    )
end;

i = 1/39:  34.343674 seconds (50.54 M allocations: 6.172 GiB, 10.15% gc time)
i = 2/39:   2.420318 seconds (688.82 k allocations: 928.100 MiB, 6.03% gc time)
i = 3/39:   3.352004 seconds (1.23 M allocations: 1.475 GiB, 6.77% gc time)
i = 4/39:   5.513699 seconds (799.84 k allocations: 1.101 GiB, 3.43% gc time)
i = 5/39:   3.330074 seconds (1.18 M allocations: 1.370 GiB, 6.36% gc time)
i = 6/39:   1.249144 seconds (427.65 k allocations: 545.157 MiB, 6.61% gc time)
i = 7/39:   6.613541 seconds (1.48 M allocations: 2.659 GiB, 5.94% gc time)
i = 8/39:   8.547364 seconds (1.50 M allocations: 2.556 GiB, 4.55% gc time)
i = 9/39:   0.716327 seconds (170.03 k allocations: 206.183 MiB, 4.73% gc time)
i = 10/39:  12.155078 seconds (1.31 M allocations: 1.925 GiB, 2.45% gc time)
i = 11/39:  10.892541 seconds (1.26 M allocations: 1.627 GiB, 2.64% gc time)
i = 12/39:   4.961075 seconds (1.07 M allocations: 1.514 GiB, 4.94% gc time)
i = 13/39:   2.839080 seconds (1.35 M allocations: 1.143 GiB, 7.06% g

In [24]:
S = map(myelindomains, sols) do m, s
    [integrate(s(t), m) for t in tspan[1]:dt:tspan[2]]
end;

In [25]:
Stotal = sum(S);

In [26]:
# Stotals = Dict{Int,Any}();
# push!(Stotals, 90 => Stotal);
# push!(Stotals, 0 => Stotal)

In [27]:
Tspan = tspan[2] - tspan[1]
@show btparams.R2_lp;
@show (-1/Tspan)*log(norm(Stotal[end])/norm(Stotal[1]));
@show exp(-tspan[end]*btparams.R2_lp);
@show norm(Stotal[end])/norm(Stotal[1]);

btparams.R2_lp = 15.873015873015873
(-1 / Tspan) * log(norm(Stotal[end]) / norm(Stotal[1])) = 16.955360374530144
exp(-(tspan[end]) * btparams.R2_lp) = 0.006223859418487457
norm(Stotal[end]) / norm(Stotal[1]) = 0.004401916257272793


In [28]:
myelin_area = intersect_area(outer_circles, bdry) - intersect_area(inner_circles, bdry)
total_area = area(bdry)
exact_mwf = myelin_area/total_area

0.2920561165866082

In [29]:
mxcall(:addpath, 0, "/home/coopar7/Documents/code/BlochTorreyExperiments-master/Experiments/MyelinWaterOrientation/MATLAB/SE_corr/");

In [36]:
MWImaps, MWIdist, MWIpart = fitmwfmodel(Stotal, NNLSRegression();
    T2Range = [10e-3, 2.0],
    spwin = [10e-3, 40e-3],
    mpwin = [41e-3, 200e-3],
    nT2 = 32,
    RefConAngle = 165.0,
    PLOTDIST = true
);
getmwf(NNLSRegression(), MWImaps, MWIdist, MWIpart)

0.27325065471209015

In [31]:
# TestStotalMagn = mwimodel(TwoPoolMagnToMagn(), tspan[1]:dt:tspan[2], modelfit.param)
# TestStotal = [Vec{2}((zero(y),y)) for y in TestStotalMagn];

In [116]:
modeltypes = (TwoPoolMagnToMagn(), ThreePoolMagnToMagn(), ThreePoolCplxToMagn(), ThreePoolCplxToCplx())
for modeltype in modeltypes
    local modelfit, errors, mwf
    println("Model: $modeltype"); flush(stdout)
    modelfit, errors = fitmwfmodel(Stotal, modeltype; TE = dt);
    mwf = getmwf(modeltype, modelfit, errors)
    println("mwf: $mwf"); flush(stdout)
    errors == nothing ? display(modelfit.param) : display([modelfit.param errors]); flush(stdout)
end

Model: TwoPoolMagnToMagn()
mwf: 0.2827439734742542


4×2 Array{Float64,2}:
  3.17098  0.00176576
  8.04404  0.00219064
 66.7613   0.079303  
 15.873    0.00254488

Model: ThreePoolMagnToMagn()
mwf: 0.2825122411749458


6×2 Array{Float64,2}:
  3.16857    0.0162875
  5.52833  638.951    
  2.5188   638.934    
 66.8205     0.269605 
 15.7308    26.4624   
 16.2055    62.4889   

Model: ThreePoolCplxToMagn()
mwf: 0.2710987416132759


8×2 Array{Float64,2}:
  3.0356      0.948621
  5.72177   601.708   
  2.44001   602.643   
 67.4323     10.1688  
 15.2282    112.529   
 18.0145    416.236   
  1.97499    55.4821  
  0.235784   57.1368  

Model: ThreePoolCplxToCplx()
mwf: 0.2817141963919488


10×2 Array{Float64,2}:
  3.16042   0.0452979  
  7.38051   1.55456    
  0.677603  1.59665    
 67.0826    0.815931   
 15.611     0.360911   
 18.9044    2.50322    
 50.2988    0.0527338  
 49.9838    0.0394197  
 50.5878    1.00425    
 -1.57074   0.000748027

In [117]:
@show getmwf(Stotal, TwoPoolMagnToMagn(); TE = dt, fitmethod = :local);
@show getmwf(Stotal, ThreePoolMagnToMagn(); TE = dt, fitmethod = :local);
@show getmwf(Stotal, ThreePoolCplxToMagn(); TE = dt, fitmethod = :local);
@show getmwf(Stotal, ThreePoolCplxToCplx(); TE = dt, fitmethod = :local);

getmwf(Stotal, TwoPoolMagnToMagn(); TE=dt, fitmethod=:local) = 0.2827439734754173
getmwf(Stotal, ThreePoolMagnToMagn(); TE=dt, fitmethod=:local) = 0.2825122411749458
getmwf(Stotal, ThreePoolCplxToMagn(); TE=dt, fitmethod=:local) = 0.2710987416132759
getmwf(Stotal, ThreePoolCplxToCplx(); TE=dt, fitmethod=:local) = 0.2817141963919488


In [108]:
p0 = initialparams(ThreePoolCplxToCplx(), ts, Stotal)[1];
modelfit, errors = fitmwfmodel(Stotal, ThreePoolCplxToCplx(); TE = dt);

In [109]:
# mwimodel(ThreePoolCplxToCplx(), ts, modelfit.param);
[mwimodel(ThreePoolCplxToCplx(), ts,  modelfit.param) |> x -> reinterpret(ComplexF64, x) complex.(Stotal)]
# [mwimodel(ThreePoolCplxToCplx(), ts, p0) |> x -> reinterpret(ComplexF64, x) complex.(Stotal)]

33×2 Array{Complex{Float64},2}:
  0.000586185+11.2185im             0.0+11.5964im  
   -0.0450738-8.48986im      -0.0451118-8.48978im  
    0.0546488+6.68979im       0.0529386+6.69008im  
   -0.0524674-5.4242im       -0.0525789-5.42419im  
    0.0471917+4.48268im       0.0486624+4.48241im  
   -0.0417105-3.75013im      -0.0434694-3.74979im  
    0.0367748+3.16134im        0.038089+3.1611im   
   -0.0324405-2.67755im      -0.0330114-2.67747im  
    0.0285982+2.2743im        0.0284292+2.2744im   
   -0.0251402-1.93516im      -0.0243914-1.9354im   
    0.0219968+1.64833im       0.0208806+1.64867im  
   -0.0191307-1.40494im      -0.0178514-1.40533im  
    0.0165239+1.19797im       0.0152494+1.19836im  
             ⋮                                     
  -0.00303313-0.287254im    -0.00366105-0.286927im 
   0.00237848+0.245198im     0.00312372+0.244812im 
  -0.00183282-0.209312im    -0.00266526-0.208878im 
   0.00138164+0.17869im      0.00227408+0.17822im  
  -0.00101174-0.152558im    -0.0

In [37]:
ts = collect(tspan[1]:dt:tspan[2])
y_biexp = @. (total_area - myelin_area) * exp(-ts*btparams.R2_lp) + myelin_area * exp(-ts*btparams.R2_sp);

In [38]:
mxcall(:figure, 0)
mxcall(:plot, 0, collect(ts), norm.(Stotal))
mxcall(:hold, 0, "on")
mxcall(:plot, 0, collect(ts), y_biexp)

In [ ]:
# prob = ODEProblem((du,u,p,t)->A_mul_B!(du,p[1],u), u0, tspan, (Amap,));

In [ ]:
#@time Expokit.expmv!(u, tspan[end], Amap, u0; tol=1e-4, norm=expmv_norm, m=100); # penelope: 17.42s
#@time Expokit.expmv!(u, tspan[end], Amap, u0; tol=1e-4, norm=expmv_norm, m=50); # penelope: 30.09s
#@time Expokit.expmv!(u, tspan[end], Amap, u0; tol=1e-4, norm=expmv_norm, m=10); # penelope: 103.5s
#@time Expokit.expmv!(u, tspan[end], Amap, u0; tol=1e-8, norm=expmv_norm); # penelope: 53.2s
#@time Expokit.expmv!(u, tspan[end], Amap, u0; tol=1e-6, norm=expmv_norm); # penelope: 44.4s

In [ ]:
#@time sol = solve(prob, CVODE_BDF(linear_solver=:GMRES); saveat=tspan, reltol=1e-8, alg_hints=:stiff); # penelope: 90.21s
#@time sol = solve(prob, CVODE_BDF(linear_solver=:GMRES); saveat=tspan, reltol=1e-4, alg_hints=:stiff); # penelope: 33.44s
#@time sol = solve(prob, CVODE_BDF(linear_solver=:BCG); saveat=tspan, reltol=1e-4, alg_hints=:stiff) # penelope: 53.66s
#@time sol = solve(prob, CVODE_BDF(linear_solver=:TFQMR); saveat=tspan, reltol=1e-4, alg_hints=:stiff) # penelope: 18.99s but low accuracy

In [ ]:
#prob_Ku = ODEProblem(K_mul_u!, u0, tspan, (K,), mass_matrix=M);
#@time sol_Ku = solve(prob_Ku, Rosenbrock23(), saveat=tspan, reltol=1e-4, alg_hints=:stiff) #DNF
#@time sol_Ku = solve(prob_Ku, Rodas4(), saveat=tspan, reltol=1e-4, alg_hints=:stiff) #DNF

In [ ]:
# @show norm(sol.u[end] - u)/maximum(abs,u);
# @show maximum(sol.u[end] - u)/maximum(abs,u);

# Testing

In [123]:
using BlochTorreyUtilsTest

## Single Axon

In [234]:
BlochTorreyUtilsTest.singleaxontests(
    BlochTorreyParameters{Float64}(
        ChiI = -60e-9,
        ChiA = -120e-9
    );
    PLOTOMEGA = false#true
);

1/1: Interior
1/1: Annular
1/1: Exterior
  1.435347 seconds (13.87 M allocations: 449.447 MiB, 22.82% gc time)
Test Summary: | Pass  Total
Single Axon   |   18     18


In [ ]:
domainsetup = BlochTorreyUtilsTest.multipleaxons();

1/20: Interior
1/20: Annular


In [ ]:
BlochTorreyUtilsTest.multipleaxontests(
    BlochTorreyParameters{Float64}(
        ChiI = -60e-9,
        ChiA = -120e-9
    ),
    domainsetup;
    PLOTOMEGA = false#true
);

### Exporting to VTK
To visualize the result we export the grid and our field `u`
to a VTK-file, which can be viewed in e.g. [ParaView](https://www.paraview.org/).

In [ ]:
# vtk_grid("bloch_torrey_equation", dh) do vtk
#     vtk_point_data(vtk, dh, u)
# end